In [2]:
from pprint import pprint

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt


import prepare as pr 

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
df = pr.prep_data('all_books .csv')

In [48]:
data = df.copy()

# NLP

# Count Vector

In [49]:
data.head(1)

,title,summary,year_published,author,review_count,number_of_ratings,length,genre,rating,reviews,cleaned_title,cleaned_summary,target,lemmatized_summary,neg,neutral,pos,compound,sentiment
0,1491,"In this groundbreaking work of science, histor...",back,Charles C. Mann,4581,81796,1492.0,History,4.04,NaN,1491,"in this groundbreaking work of science, histor...",unsuccessful,groundbreaking science history archaeology cha...,0.021,0.848,0.132,0.9393,very positive


In [ ]:
#  document= 'mary had a '

In [62]:
document = ' '.join(data.lemmatized_summary)

In [66]:
# Bag of Words
words = pd.Series(document.split())

# From the Series we can extract the value_counts, which is our raw count
# for term frequency. Once we have the raw counts, we can calculate the
# other measures.
(pd.DataFrame({'raw_count': words.value_counts()})
 .assign(frequency=lambda df: df.raw_count / df.raw_count.sum())
 .assign(augmented_frequency=lambda df: df.frequency / df.frequency.max()))


,raw_count,frequency,augmented_frequency
life,2595,0.007344,1.000000
one,2482,0.007024,0.956455
world,1870,0.005292,0.720617
find,1655,0.004683,0.637765
love,1621,0.004587,0.624663
...,...,...,...
herselfshe,1,0.000003,0.000385
romancedespising,1,0.000003,0.000385
companyincluding,1,0.000003,0.000385
highbrow,1,0.000003,0.000385


In [63]:
def idf(word):
    n_occurrences = sum([1 for doc in documents.values() if word in doc])
    return math.log(len(documents) / n_occurrences)

In [40]:
'''def idf(word):
    # Count the number of documents containing the word
    doc_count = sum(1 for doc in documents.values() if word in doc)
    # Calculate idf
    idf = math.log(len(documents) / doc_count)
    return idf'''


'def idf(word):\n    # Count the number of documents containing the word\n    doc_count = sum(1 for doc in documents.values() if word in doc)\n    # Calculate idf\n    idf = math.log(len(documents) / doc_count)\n    return idf'

In [64]:
tfs = []

# We'll caclulate the tf-idf value for every word across every document

# Start by iterating over all the documents
for doc, text in documents.items():
    # We'll make a data frame that contains the tf for every word in every document
    df = (pd.Series(text.split())
          .value_counts()
          .reset_index()
          .set_axis(['word', 'raw_count'], axis=1, inplace=False)
          .assign(tf=lambda df: df.raw_count / df.shape[0])
          .drop(columns='raw_count')
          .assign(doc=doc))
    # Then add that data frame to our list
    tfs.append(df)

# We'll then concatenate all the tf values together.
(pd.concat(tfs)
 # calculate the idf value for each word
 .assign(idf=lambda df: df.word.apply(idf))
 # then use the if and idf values to calculate tf-idf 
 .assign(tf_idf=lambda df: df.idf * df.tf)
 .drop(columns=['tf', 'idf'])
 .sort_values(by='tf_idf', ascending=False))

/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_30939/2599928771.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = (pd.Series(text.split())
/var/folders/d0/5dr5q0n145vcn3_m9dxy1xq80000gn/T/ipykernel_30939/2599928771.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = (pd.Series(text.split())


TypeError: 'numpy.ndarray' object is not callable

In [44]:
(pd.concat(tfs)
 # calculate the idf value for each word
 .assign(idf=lambda df: df.word.apply(idf))
 # then use the if and idf values to calculate tf-idf 
 .assign(tf_idf=lambda df: df.idf * df.tf)
 .drop(columns=['tf', 'idf'])
 .sort_values(by='tf_idf', ascending=False)
 .pipe(lambda df: pd.crosstab(df.doc, df.word, values=df.tf_idf, aggfunc=lambda x: x))
 .fillna(0))

,word,tf,doc
0,war,0.046729,3809
1,american,0.028037,3809
2,life,0.028037,3809
3,english,0.028037,3809
4,great,0.018692,3809
...,...,...,...
102,tide,0.009346,3809
103,nazi,0.009346,3809
104,engulf,0.009346,3809
105,parent,0.009346,3809
